In [ ]:
import os
import time
import json
import math
import requests
import pandas as pd
from typing import Dict, Any, List, Optional

BASE_URL = "https://api.seatgeek.com/2/events"

def fetch_page(
    client_id: str,
    start_utc: str,
    end_utc: str,
    page: int,
    per_page: int = 100,
    extra_params: Optional[Dict[str, Any]] = None,
    max_retries: int = 5,
    backoff_seconds: float = 1.5,
) -> Dict[str, Any]:
    """
    Descarga una página de eventos usando filtros por datetime_utc y paginación.
    """
    params = {
        "client_id": client_id,
        "page": page,
        "per_page": per_page,
        "datetime_utc.gte": start_utc,
        "datetime_utc.lte": end_utc,
    }
    if extra_params:
        params.update(extra_params)

    last_err = None
    for attempt in range(1, max_retries + 1):
        try:
            r = requests.get(BASE_URL, params=params, timeout=30)

            if r.status_code in (429, 502, 503, 504):
                sleep_s = backoff_seconds * attempt
                time.sleep(sleep_s)
                continue

            r.raise_for_status()
            return r.json()

        except Exception as e:
            last_err = e
            time.sleep(backoff_seconds * attempt)

    raise RuntimeError(f"Fallo descargando page={page}. Último error: {last_err}")


def extract_events_2025(
    client_id: str,
    out_json_path: str = "seatgeek_events_2025.json",
    out_csv_path: str = "seatgeek_events_2025.csv",
    per_page: int = 100,
):
    # 2025-01-01 hasta fin de año
    start_utc = "2025-01-01T00:00:00"
    end_utc = "2025-12-31T23:59:59"

    all_events: List[Dict[str, Any]] = []

    # Primera pagina para saber el total
    first = fetch_page(
        client_id=client_id,
        start_utc=start_utc,
        end_utc=end_utc,
        page=1,
        per_page=per_page,
    )

    meta = first.get("meta", {}) or {}
    total = meta.get("total", 0) or 0
    total_pages = int(math.ceil(total / per_page)) if total else 1

    events = first.get("events", []) or []
    all_events.extend(events)

    print(f"Total eventos (según meta.total): {total}")
    print(f"Páginas estimadas: {total_pages} (per_page={per_page})")
    print(f"Página 1 -> eventos: {len(events)}")

    # Itero sobre el resto de pags
    for page in range(2, total_pages + 1):
        data = fetch_page(
            client_id=client_id,
            start_utc=start_utc,
            end_utc=end_utc,
            page=page,
            per_page=per_page,
        )
        events = data.get("events", []) or []
        all_events.extend(events)
        print(f"Página {page} -> eventos: {len(events)} (acumulados: {len(all_events)})")

        time.sleep(0.2)

    # Guardo el JSON
    with open(out_json_path, "w", encoding="utf-8") as f:
        json.dump(all_events, f, ensure_ascii=False, indent=2)

    # 4) Guardo CSV con los campos utiles
    rows = []
    for ev in all_events:
        venue = ev.get("venue") or {}
        performers = ev.get("performers") or []
        rows.append({
            "id": ev.get("id"),
            "title": ev.get("title"),
            "type": ev.get("type"),
            "datetime_utc": ev.get("datetime_utc"),
            "url": ev.get("url"),
            "score": ev.get("score"),
            "venue_name": venue.get("name"),
            "venue_city": venue.get("city"),
            "venue_state": venue.get("state"),
            "venue_country": venue.get("country"),
            "venue_lat": venue.get("lat"),
            "venue_lon": venue.get("lon"),
            "performers": ", ".join([p.get("name", "") for p in performers if p.get("name")]),
        })

    df = pd.DataFrame(rows)
    df.to_csv(out_csv_path, index=False, encoding="utf-8")

    print("\n Listo")
    print(f"- JSON: {out_json_path} ({len(all_events)} eventos)")
    print(f"- CSV:  {out_csv_path} ({len(df)} filas)")

if __name__ == "__main__":

    client_id = os.getenv("SEATGEEK_CLIENT_ID")
    
    if not client_id:
        raise SystemExit("❌ Falta SEATGEEK_CLIENT_ID. Define la variable de entorno o pega el client_id en el script.")

    extract_events_2025(client_id=client_id)
